In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
dataset_dir = '/content/drive/MyDrive/10 DMC/X-Ray'
# Check if the path exists
if os.path.exists(dataset_dir):
    print("Dataset directory exists.")
    print("Contents of dataset_dir:", os.listdir(dataset_dir))
    print("Contents of train folder:", os.listdir(os.path.join(dataset_dir, 'train')))
else:
    print("Dataset directory does NOT exist. Check the path.")


Dataset directory does NOT exist. Check the path.


In [ ]:
# Required imports
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import f1_score, confusion_matrix
import os
# Mount Google Drive
drive.mount('/content/drive')

# Dataset directory
dataset_dir = '/content/drive/MyDrive/10 DMC/X-Ray'

# Image size and batch size
img_size = (224, 224)
batch_size = 32

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescale for testing
test_datagen = ImageDataGenerator(rescale=1./255)

# Load training data
train_generator = train_datagen.flow_from_directory(
    directory=dataset_dir + '/train',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load testing data
test_generator = test_datagen.flow_from_directory(
    directory=dataset_dir + '/test',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Verify dataset loading
print("Classes in training data:", train_generator.class_indices)
print("Number of training samples:", train_generator.samples)
print("Number of testing samples:", test_generator.samples)
'''
# Data Preprocessing and Dataset Loading
# Assuming you have a dataset with images stored in directories like 'COVID19', 'NORMAL', 'PNEUMONIA'
dataset_dir =  '/content/drive/MyDrive/10 DMC/X-RAY'  # Replace 'Dataset' with the actual folder name in your Drive
 # Change this to your dataset path
img_size = (224, 224)  # Image size for model input

# Use ImageDataGenerator for image loading and augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True,
                                   fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(os.path.join(dataset_dir, 'train'), target_size=img_size, batch_size=32, class_mode='categorical')
val_data = test_datagen.flow_from_directory(os.path.join(dataset_dir, 'val'), target_size=img_size, batch_size=32, class_mode='categorical')
test_data = test_datagen.flow_from_directory(os.path.join(dataset_dir, 'test'), target_size=img_size, batch_size=32, class_mode='categorical')

'''

# Model Definitions (4 models as examples)
def Create_model(input_size=(224, 224, 3)):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_size),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(3, activation='softmax')  # Change '3' to the number of classes you have
    ])
    return model

def Create_model_2(input_size=(224, 224, 3)):
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=input_size),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(3, activation='softmax')  # Change '3' to the number of classes you have
    ])
    return model

def Create_model_3(input_size=(224, 224, 3)):
    model = Sequential([
        Conv2D(128, (3, 3), activation='relu', input_shape=input_size),
        MaxPooling2D(2, 2),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(256, activation='relu'),
        Dense(3, activation='softmax')  # Change '3' to the number of classes you have
    ])
    return model

def Create_model_4(input_size=(224, 224, 3)):
    model = Sequential([
        Conv2D(256, (3, 3), activation='relu', input_shape=input_size),
        MaxPooling2D(2, 2),
        Conv2D(512, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(3, activation='softmax')  # Change '3' to the number of classes you have
    ])
    return model

# Compile the models
def model_compiling(model, loss='categorical_crossentropy', optimizer='adam'):
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train models and save their weights
model1 = Create_model()
model_compiling(model1)
model2 = Create_model_2()
model_compiling(model2)
model3 = Create_model_3()
model_compiling(model3)
model4 = Create_model_4()
model_compiling(model4)

# Define callbacks
model_path_1 = "Model_1.h5"
checkpoint_1 = ModelCheckpoint(filepath=model_path_1, save_weights_only=True, monitor='val_accuracy', save_best_only=True, verbose=1)
earlystop = EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=6, factor=0.2, verbose=1, min_lr=0.00000001)

# Train model1 and save weights
history1 = model1.fit(train_data, validation_data=val_data, epochs=4, callbacks=[earlystop, checkpoint_1, learning_rate_reduction])

# Similarly, train the other models (model2, model3, model4), and save their weights
history2 = model2.fit(train_data, validation_data=val_data, epochs=4, callbacks=[earlystop, checkpoint_1, learning_rate_reduction])
history3 = model3.fit(train_data, validation_data=val_data, epochs=4, callbacks=[earlystop, checkpoint_1, learning_rate_reduction])
history4 = model4.fit(train_data, validation_data=val_data, epochs=4, callbacks=[earlystop, checkpoint_1, learning_rate_reduction])

# Save model weights after training
model1.save_weights('./model1_weights.h5')
model2.save_weights('./model2_weights.h5')
model3.save_weights('./model3_weights.h5')
model4.save_weights('./model4_weights.h5')

# Evaluate the models on the test data
model1.evaluate(test_data)
model2.evaluate(test_data)
model3.evaluate(test_data)
model4.evaluate(test_data)

# Confusion Matrix for evaluation
predictions = model1.predict(test_data)
conf_mat = confusion_matrix(test_data.labels, predictions)
print(conf_mat)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/10 DMC/X-Ray/train'

In [ ]:
# Required imports
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained models (do not retrain)
model1 = Create_model()
model2 = Create_model_2()
model3 = Create_model_3()
model4 = Create_model_4()

# Load saved weights
model1.load_weights('./model1_weights.h5')
model2.load_weights('./model2_weights.h5')
model3.load_weights('./model3_weights.h5')
model4.load_weights('./model4_weights.h5')

# Function to preprocess a single image
def preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize the image
    return img_array

# Function to make predictions on a new image
def predict_image(model, img_path):
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)  # Get the class with the highest probability
    return predicted_class

# Test the models on new images
img_path = 'path_to_your_image.jpg'  # Change this to the path of your image

pred1 = predict_image(model1, img_path)
pred2 = predict_image(model2, img_path)
pred3 = predict_image(model3, img_path)
pred4 = predict_image(model4, img_path)

# Print out predictions
print(f"Model 1 Prediction: {pred1}")
print(f"Model 2 Prediction: {pred2}")
print(f"Model 3 Prediction: {pred3}")
print(f"Model 4 Prediction: {pred4}")

# Accuracy: Assuming you have ground truth (true class) for your image
true_class = 1  # Replace with actual class of your image (0: COVID, 1: NORMAL, 2: PNEUMONIA)
accuracy1 = (pred1 == true_class)
accuracy2 = (pred2 == true_class)
accuracy3 = (pred3 == true_class)
accuracy4 = (pred4 == true_class)

print(f"Accuracy for Model 1: {accuracy1}")
print(f"Accuracy for Model 2: {accuracy2}")
print(f"Accuracy for Model 3: {accuracy3}")
print(f"Accuracy for Model 4: {accuracy4}")
